# L-systems


In [2]:
from pgljupyter import *

## Signals

Complete the following model  to simulate the propagation of a signal



In [3]:
%%lpy -w 50 -a True

Delay = 5
LDelay = 10
T = 120

Axiom: R(T) I(0) A(Delay, 0)

derivation length: 150
production:
consider: R I A B

I(x) < A(d,o) :
  if x == 1 : produce W   # produce flower
  elif d > 0: produce A(d-1, o) # continue to growth
  else:
    if o == 0: nproduce [ +(60) A(0,o+1) ] # produce lateral apex
    else : nproduce [ +(60) /(60) ,(2) ~l ] # or lateral leaf
    produce I(0) /(180)  A(Delay if o == 0 else LDelay, o)

R(t) --> R(t-1)



interpretation:

W --> _(0.3) ,(3) @O
I(x) --> ,(2 if x ==0 else 4) _(0.1 if x ==0 else 0.2) F

endlsystem


<string>:13: Warning: Found symbol '#' after Lstring. Considered as begining of comments
<string>:14: Warning: Found symbol '#' after Lstring. Considered as begining of comments
<string>:16: Warning: Found symbol '#' after Lstring. Considered as begining of comments
<string>:17: Warning: Found symbol '#' after Lstring. Considered as begining of comments


LsystemWidget(animate=True, derivationLength=151, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe…

### Modelling Fluxes

Introduce a destruction ratio
Change the source of creation ot the middle 

In [9]:
%%lpy -w 100 -a True

transport_speed = 1/2.

nbEntity = 10

min_level = 0.01
max_level = 10

def txt(val):
  return str(round(val,2))[:5]

Axiom: 
  nproduce _(2) 
  for i in range(nbEntity-1):
    nproduce I(min_level)
  nproduce I(max_level)

derivation length: 200
production:

I(a) < I(p) > I(b) :
  produce I(p + (a+b-2*p)*transport_speed)


I(p) > I(a):
  produce I(p+ (a-p)*transport_speed)

I(a) < I(p) :
  produce I(p)


interpretation:

I(a) :
    alpha =  (a-min_level)/(max_level-min_level)
    a += 0.1
    produce InterpolateColors(4,3, alpha)F(3)[-(90)f(5)_(1)F(a)]F(3)

endlsystem


LsystemWidget(animate=True, derivationLength=201, is_magic=True, scene={'data': b'x\xda\x8d\x96Mk\x13A\x18\x80…

## Environment

Test the root model : [grid-rootinsoil2.lpy](./env/grid-rootinsoil2.lpy) with [grid.py](./env/grid.py)

Test the light model : [light-growth.lpy](./env/light-growth.lpy) with [light.py](./env/sunDome.py) and [sunDome.py](./env/light.py)

## Arborescence

A partir d'une structure binaire et planaire [binarytree.lpy ](./binarytree.lpy ), générer une structure 3D avec un nombre aléatoire de ramifications compris entre 2 et 4.

Ajouter de l'heliotropisme.

In [ ]:
%%lpy -w 10
Axiom: A

derivation length: 6
production:

A --> F[+(30) A][-(30) A]P

interpretation:

P --> [,(3) @O(0.15) ]
endlsystem

## Space Colonization

Tester la generation d'un nuage de point avec [pointinprofile.lpy](./pointinprofile.lpy)

Completer l'algorithme de colonization de l'espace [spacecolonization-canevas.lpy](./spacecolonization-canevas.lpy)

In [ ]:
%%lpy -w 30

from openalea.plantgl.all import *
from random import uniform
from math import pi,radians

profile = QuantisedFunction(NurbsCurve2D([(0, 0.0181818, 1),
         (0.219697, 0.0363636, 1),(0.297727, 0.313636, 1),(0.389394, 0.327273, 1),
         (0.593182, 0.290909, 1),(0.688636, 0.0590909, 1),(0.784091, 0.0954545, 1),
         (0.884091, 0.172727, 1),(1, 0, 1)] ))

def generate_points(nbpoints,size):
  pts = []
  for i in range(nbpoints):
    angle = uniform(0,2 * pi)
    ok = False
    while not ok:
      x,y,z =  uniform(-1,1),uniform(-1,1),uniform(0,1)
      if (x**2+y**2) <= profile(z)**2: ok = True
    pts.append(Vector3(x,y,z)*size)
  return pts

pts = generate_points(5000,20)

scale = 1
grow_dist = 1 * scale
killradius = grow_dist * 1
perceptionradius = grow_dist * 1.5 
min_nb_pt = 5
rootpos = Vector3(0,0,1)

# place the attractor points into a grid
attractor_grid = Point3Grid(pts, 10)

rootattr = attractor_grid.query_ball_point(rootpos, perceptionradius)
attractor_grid.disable_points(attractor_grid.query_ball_point(rootpos, killradius) )

backward()

initial_radius = 0.04
e = 2.5

def piperadius(ris):
   return pow(sum([pow(ri,e) for ri in ris]),1./e)

module Bud, I, Node, Attractors

Axiom: Attractors @Gc Bud(rootpos, rootattr)

derivation length: 100
production:


Bud(pt, atts) :  
       # compute mean direction    
       # compute new position
       # remove closest attractors
       # produce active nodes    
       pass

Node(pt):
  # find nearest attractor points in cone of perception of given radius and angle
  # if enough attractors are available, produce new bud
  pass

I(pt, r) >> x([I(lpt, lr)]) :
  if len(lr) >= 1:
    produce I(pt,piperadius(lr))

interpretation:

I(pos,radius) --> _(radius) LineTo(pos)
Bud(pt, atts) -->  [SetColor(5) Sphere(0.1*scale)]
Node(pt) -->  [SetColor(3) Sphere(0.1*scale)]

Attractors :
  pttodisplay = attractor_grid.get_enabled_points()
  if len(pttodisplay) > 0:
    nproduce [,(3)
    for p in pttodisplay:
        nproduce @M(p) Sphere(0.05) 
    produce ]
endlsystem